<font size="6"> <center> <span style="color:SteelBlue"> <b> Information Extraction and Structuring Unstructured Data</b></center></font>

ref: https://towardsdatascience.com/something-from-nothing-use-nlp-and-ml-to-extract-and-structure-web-data-3f49b2f72b13

> The goal of this study is to extract the following items from 510 legal contract documents, and structure the information extracted in a tabular format.

        Name of parties involved in the contract
        Place of the contract
        Date of contract 
        Contract value (Amount of money)
        Contract duration



# Data Import from txt Files

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

> We first define a custom function, called `read_mpl_txt_files`, that reads all the txt files, and places each txt file in a row of a dataframe.

In [2]:
def read_mpl_txt_files(folder_path):
    import os, glob
    import csv

    files = []   # will be a list of DataFrames, where each text file's paragraph is a row of the DF
    files = [
        pd.read_csv(
            file,
            header=None,
            delimiter='\t',             # tab-delimited file
            quoting=csv.QUOTE_NONE)     # to address error: "Error tokenizing data. C error: EOF inside string starting at row 486"
        for file in glob.glob(os.path.join(folder_path, '*.txt'))
    ]

    txt_files = []   # list that has all the text of a single document as a string list member
    for df_txt in files:
        df_txt.fillna('empty', inplace=True)
        txt_files.append(' '.join(df_txt[0]))
    df = pd.DataFrame(txt_files)
    return df

In [3]:
folder_path = '/home/sam/my_files/Springboard_Data_Analytics/13-NLP/Omdena_NLP_Course/Week4_NER/Week4_Project'

df = read_mpl_txt_files(folder_path)
df.columns = ['text']
print(df.shape)
df.head()

(510, 1)


text
0  Exhibit 10.5 Reseller Agreement This Agreement...
1  AMENDMENT NO. 3 Dated as of February 19, 2007 ...
2  Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...
3  Exhibit 99.4 JOINT FILING AGREEMENT In accorda...
4  Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...

# Contract Place

> Place of contract (principal place of business for the parties involved in the contract) is typically stated in the first few paragraphs of the contract. So, we will look at the first half of each document using "Sentence Segmentation" from SapCy. We then use `locationtagger` in two cusmton functions ("get_country" and "get_city") to organize countries and cities in two separate columns of the DataFrame.

> The main advantage of this approach is that we minimize selection of non-relevant countries or cities (e.g. locations that could be mentioned as footnotes, or at the end of the document as appendix).

In [4]:
import math
import nltk
import spacy
import locationtagger
# essential entity models downloads
# nltk.downloader.download('maxent_ne_chunker')
# nltk.downloader.download('words')
# nltk.downloader.download('treebank')
# nltk.downloader.download('maxent_treebank_pos_tagger')
# nltk.downloader.download('punkt')
# nltk.download('averaged_perceptron_tagger')

nlp = spacy.load("en_core_web_sm")


def get_country(content):
    doc = nlp(content)
    # list comprehension for getting all sentences using doc.sents from spacy
    all_sentences = [sent.text for sent in doc.sents]

    # list comprehension for extracting the first half of the document
    first_half_sentences = [
        all_sentences[i] for i in range(math.ceil(len(all_sentences) / 2))
    ]
    new_content = ' '.join(first_half_sentences)

    # Using "locationtagger" to get countries (As a list)
    countries = locationtagger.find_locations(text=new_content).countries

    return ', '.join(countries)

> We first extract country names using our custom function `get_country`:

In [5]:
df['contract_countries'] = df['text'].map(get_country)

> The `locationtagger` has a `.cities` method that finds the cities in a given text, but it also returns words that are not city names (not 100% foolproof), as cen be seen below for a given data point.

> That is why that we will apply NER on the result of `locationtagger` and only extract GPE entities.

In [6]:
# Check the get_city() custom function
locationtagger.find_locations(text=df['text'][2]).cities

['Beijing',
 'Stone',
 'Law',
 'Republic',
 'Hong Kong',
 'Agency',
 'China',
 'May',
 'Macau',
 'Sec',
 'Content',
 'Section']

In [7]:
def get_city(content):
    doc = nlp(content)
    # list comprehension for getting all sentences using doc.sents from spacy
    all_sentences = [sent.text for sent in doc.sents]

    # list comprehension for extracting the first half of the document
    first_half_sentences = [
        all_sentences[i] for i in range(math.ceil(len(all_sentences) / 2))
    ]
    new_content = ' '.join(first_half_sentences)

    # Using "locationtagger" to get cities (as a list)
    cities = locationtagger.find_locations(text=new_content).cities

    # the .cities method above from locationtagger sometimes returns other words than city names, so, we will refine it by extracting NER's GPE entities
    doc = nlp(', '.join(cities))
    cities_ner = [X.text for X in doc.ents if X.label_ == 'GPE']

    return ', '.join(cities_ner)

In [8]:
doc = nlp(get_city(df['text'][2]))
', '.join([X.text for X in doc.ents if X.label_ == 'GPE'])

'Beijing, Law, Republic, Hong Kong, Agency, China, May, Macau'

> We can see that for the index row of 2, our custom function eliminated most non-city names.

In [9]:
df['contract_cities'] = df['text'].map(get_city)

> As can be seen below, in some cases the city column includes a country name that is present in the country column; so to make the above approach more robust, we will check the cities column for each row against the country column, and remove duplicate country names from the city column.

In [10]:
df

text  \
0    Exhibit 10.5 Reseller Agreement This Agreement...   
1    AMENDMENT NO. 3 Dated as of February 19, 2007 ...   
2    Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...   
3    Exhibit 99.4 JOINT FILING AGREEMENT In accorda...   
4    Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...   
..                                                 ...   
505  1                                             ...   
506  Exhibit 10.16   ENDORSEMENT AGREEMENT     THIS...   
507  Exhibit 10.1 CERTAIN CONFIDENTIAL PORTIONS OF ...   
508  Strategic Alliance Agreement   This agreement ...   
509  1                                             ...   

                   contract_countries  \
0                     Romania, Canada   
1                                       
2    China, Cayman Islands, Hong Kong   
3                                       
4                       United States   
..                                ...   
505                                     
506                     United States   
507                           Bermuda   
508                                     
509                                     

                                       contract_cities  
0                                    Vancouver, Canada  
1                                                       
2    Beijing, Law, Republic, Hong Kong, Agency, Chi...  
3                                             Delaware  
4       Melbourne, California, Roseville, Florida, Usa  
..                                                 ...  
505                                            Florida  
506                                            America  
507       Watford, Fleet, Purchase, Canon, England, Eu  
508                   Rancho Cordova, Nevada, Colorado  
509                Irvine, California, San Luis Obispo  

[510 rows x 3 columns]

In [11]:
def remove_duplicate(text1, text2):
    """
    >>> Inpouts: two string columns of a DataFrame: text1, text2
    >>> remove duplicate items (between text1 and text2) from text2
    """
    
    # split the content of each cell based on "," separator and white spacce ' '
    text1 = str(text1).split(', ')
    text2 = str(text2).split(', ')
    unwanted_words = []

    for word in text2:
        # check to see if the lower case "word" from text2 is in text1 (with all text1 members converted to lower case)
        if word.lower() in list(map(lambda x: x.lower(), text1)):
            unwanted_words.append(word)
    
    # select words from "text" that are NOT in unwanted_words
    text2 = [ele for ele in text2 if ele not in unwanted_words]
    return ', '.join(text2)

In [12]:
df['contract_cities_clean'] = df.apply(lambda x: remove_duplicate(x['contract_countries'], x['contract_cities']), axis=1)

In [13]:
df.head(10)

text  \
0  Exhibit 10.5 Reseller Agreement This Agreement...   
1  AMENDMENT NO. 3 Dated as of February 19, 2007 ...   
2  Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...   
3  Exhibit 99.4 JOINT FILING AGREEMENT In accorda...   
4  Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...   
5  Exhibit 10.1 Strategic Alliance Agreement AGRE...   
6  Exhibit 10.1 Sales, Marketing, Distribution, a...   
7  EXHIBIT 10.8 DISTRIBUTOR AGREEMENT         Tri...   
8  Exhibit (8)(k)(k) SERVICE AGREEMENT This Agree...   
9  TAT-14 CABLE NETWORK                      CONS...   

                                  contract_countries  \
0                                    Romania, Canada   
1                                                      
2                   China, Cayman Islands, Hong Kong   
3                                                      
4                                      United States   
5                                      United States   
6  Kingdom of Saudi Arabia, Saudi Arabia, Sultana...   
7                                                      
8                                                      
9  Netherlands, France, Denmark, Germany, United ...   

                                     contract_cities  \
0                                  Vancouver, Canada   
1                                                      
2  Beijing, Law, Republic, Hong Kong, Agency, Chi...   
3                                           Delaware   
4     Melbourne, California, Roseville, Florida, Usa   
5                    New York, Cherry Hill, Maryland   
6  London, Philadelphia, West, New Brunswick, Del...   
7                                        Santa Clara   
8                                                      
9  Norden, Bonn, Point, America, Usa, Denmark, In...   

                               contract_cities_clean  
0                                          Vancouver  
1                                                     
2         Beijing, Law, Republic, Agency, May, Macau  
3                                           Delaware  
4     Melbourne, California, Roseville, Florida, Usa  
5                    New York, Cherry Hill, Maryland  
6  London, Philadelphia, West, New Brunswick, Del...  
7                                        Santa Clara  
8                                                     
9  Norden, Bonn, Point, America, Usa, Inland, Kat...

> The dulicate country names are now removed from city column (e.g. "Canada" is removed for the first row; or "Hong Kong" and "China" are removed from the third row) :)

> The last refining step is to remove any variations of the US country from the cities column; i.e. removing ['america', 'us', 'usa', 'u.s.', 'u.s.a.']

In [14]:
df['contract_cities_clean'].head(10)

0                                            Vancouver
1                                                     
2           Beijing, Law, Republic, Agency, May, Macau
3                                             Delaware
4       Melbourne, California, Roseville, Florida, Usa
5                      New York, Cherry Hill, Maryland
6    London, Philadelphia, West, New Brunswick, Del...
7                                          Santa Clara
8                                                     
9    Norden, Bonn, Point, America, Usa, Inland, Kat...
Name: contract_cities_clean, dtype: object

In [15]:
to_remove = ['america', 'us', 'usa', 'u.s.', 'u.s.a.']

def remove_us_words(text):
    text = str(text).split(', ')
    unwanted_words = []

    for word in text:
        # collect unwanted_words
        if word.lower() in to_remove:
            unwanted_words.append(word)
    
    # select words from "text" that are NOT in unwanted_words
    text = [ele for ele in text if ele not in unwanted_words]
    return ', '.join(text)

In [16]:
df['contract_cities_final'] = df['contract_cities_clean'].map(remove_us_words)

In [17]:
df.head(10)

text  \
0  Exhibit 10.5 Reseller Agreement This Agreement...   
1  AMENDMENT NO. 3 Dated as of February 19, 2007 ...   
2  Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...   
3  Exhibit 99.4 JOINT FILING AGREEMENT In accorda...   
4  Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...   
5  Exhibit 10.1 Strategic Alliance Agreement AGRE...   
6  Exhibit 10.1 Sales, Marketing, Distribution, a...   
7  EXHIBIT 10.8 DISTRIBUTOR AGREEMENT         Tri...   
8  Exhibit (8)(k)(k) SERVICE AGREEMENT This Agree...   
9  TAT-14 CABLE NETWORK                      CONS...   

                                  contract_countries  \
0                                    Romania, Canada   
1                                                      
2                   China, Cayman Islands, Hong Kong   
3                                                      
4                                      United States   
5                                      United States   
6  Kingdom of Saudi Arabia, Saudi Arabia, Sultana...   
7                                                      
8                                                      
9  Netherlands, France, Denmark, Germany, United ...   

                                     contract_cities  \
0                                  Vancouver, Canada   
1                                                      
2  Beijing, Law, Republic, Hong Kong, Agency, Chi...   
3                                           Delaware   
4     Melbourne, California, Roseville, Florida, Usa   
5                    New York, Cherry Hill, Maryland   
6  London, Philadelphia, West, New Brunswick, Del...   
7                                        Santa Clara   
8                                                      
9  Norden, Bonn, Point, America, Usa, Denmark, In...   

                               contract_cities_clean  \
0                                          Vancouver   
1                                                      
2         Beijing, Law, Republic, Agency, May, Macau   
3                                           Delaware   
4     Melbourne, California, Roseville, Florida, Usa   
5                    New York, Cherry Hill, Maryland   
6  London, Philadelphia, West, New Brunswick, Del...   
7                                        Santa Clara   
8                                                      
9  Norden, Bonn, Point, America, Usa, Inland, Kat...   

                               contract_cities_final  
0                                          Vancouver  
1                                                     
2         Beijing, Law, Republic, Agency, May, Macau  
3                                           Delaware  
4          Melbourne, California, Roseville, Florida  
5                    New York, Cherry Hill, Maryland  
6  London, Philadelphia, West, New Brunswick, Del...  
7                                        Santa Clara  
8                                                     
9               Norden, Bonn, Point, Inland, Katwijk

In [51]:
df[df['contract_cities_final'] == ''].shape[0]

76

> 76 contracts do not mention a city name (Assuming the applied algorithm finds all mentions of city names).

In [50]:
df[df['contract_countries'] == ''].shape[0]

228

> 228 contracts do not mention a country name (Assuming the applied algorithm finds all mentions of country names).

# Parties Involved

> The parties inlvolved is generally stated in the first sentence of the contract. So, we will create a custom function, `get_parties`, that uses regular expression (regex) to find the text within "between ... and ... .", and then apply NER to extract ORG entities.

> Here are the techniques used for our regex (an example is shown below for the first document of our DataFrame:

           .+ ---> one or more characters
           ?  ---> non-greedy matching (shortest match)
           () ---> group and capture

In [20]:
import nltk
import spacy

nlp = spacy.load("en_core_web_sm")

In [21]:
import re
doc = nlp(re.search(r'between(.+?)and(.+?)\.', df['text'][0]).group(0))

doc.ents

[X.text for X in doc.ents if X.label_ == 'ORG']

['B.C. Ltd.', 'Galaxy Telnet SRL']

In [22]:
def get_parties(content):
    if re.findall(r'between(.+?)and(.+?)\.', content) != []:
        doc = nlp(re.search(r'between(.+?)and(.+?)\.', content).group(0))
        parties = [X.text for X in doc.ents if X.label_ == 'ORG']
        return ' - '.join(set(parties))
    else:
        return 'NA'

In [23]:
df['parties'] = df['text'].map(get_parties)

In [24]:
df.head(12)

text  \
0   Exhibit 10.5 Reseller Agreement This Agreement...   
1   AMENDMENT NO. 3 Dated as of February 19, 2007 ...   
2   Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...   
3   Exhibit 99.4 JOINT FILING AGREEMENT In accorda...   
4   Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...   
5   Exhibit 10.1 Strategic Alliance Agreement AGRE...   
6   Exhibit 10.1 Sales, Marketing, Distribution, a...   
7   EXHIBIT 10.8 DISTRIBUTOR AGREEMENT         Tri...   
8   Exhibit (8)(k)(k) SERVICE AGREEMENT This Agree...   
9   TAT-14 CABLE NETWORK                      CONS...   
10  Exhibit 10.10 CERTAIN CONFIDENTIAL PORTIONS OF...   
11  Exhibit 26 JOINT FILING AGREEMENT Pursuant to ...   

                                   contract_countries  \
0                                     Romania, Canada   
1                                                       
2                    China, Cayman Islands, Hong Kong   
3                                                       
4                                       United States   
5                                       United States   
6   Kingdom of Saudi Arabia, Saudi Arabia, Sultana...   
7                                                       
8                                                       
9   Netherlands, France, Denmark, Germany, United ...   
10              Israel, Mexico, Canada, United States   
11                                                      

                                      contract_cities  \
0                                   Vancouver, Canada   
1                                                       
2   Beijing, Law, Republic, Hong Kong, Agency, Chi...   
3                                            Delaware   
4      Melbourne, California, Roseville, Florida, Usa   
5                     New York, Cherry Hill, Maryland   
6   London, Philadelphia, West, New Brunswick, Del...   
7                                         Santa Clara   
8                                                       
9   Norden, Bonn, Point, America, Usa, Denmark, In...   
10  Cambridge, New York, Mexico, Transfer, Section...   
11                                           Delaware   

                                contract_cities_clean  \
0                                           Vancouver   
1                                                       
2          Beijing, Law, Republic, Agency, May, Macau   
3                                            Delaware   
4      Melbourne, California, Roseville, Florida, Usa   
5                     New York, Cherry Hill, Maryland   
6   London, Philadelphia, West, New Brunswick, Del...   
7                                         Santa Clara   
8                                                       
9   Norden, Bonn, Point, America, Usa, Inland, Kat...   
10   Cambridge, New York, Transfer, Section, Michigan   
11                                           Delaware   

                                contract_cities_final  \
0                                           Vancouver   
1                                                       
2          Beijing, Law, Republic, Agency, May, Macau   
3                                            Delaware   
4           Melbourne, California, Roseville, Florida   
5                     New York, Cherry Hill, Maryland   
6   London, Philadelphia, West, New Brunswick, Del...   
7                                         Santa Clara   
8                                                       
9                Norden, Bonn, Point, Inland, Katwijk   
10   Cambridge, New York, Transfer, Section, Michigan   
11                                           Delaware   

                                              parties  
0                       Galaxy Telnet SRL - B.C. Ltd.  
1   Glu Mobile Inc. - Administrator for Twentieth ...  
2   Beijing SINA Internet Information Service Co.,...  
3                                                  NA  
4   Second Floor - Erchonia Corporation - InnerSco...  
5  

In [52]:
df[df['parties'] == 'NA'].shape[0]

30

> The parties for 30 contracts were not identified by this method; either becauase the parteis are not specified, or becauase they are specified in a different textual structure from "between ... and ...".

# Effective Date

> Let's see how the NER DATE entity performs for extracting date information:

In [26]:
df['text'][0]

'Exhibit 10.5 Reseller Agreement This Agreement is made and entered into by and between 695014 B.C. Ltd. dba Galaxy Telecom, having a principal office at 200 - 375 Water Street, Vancouver, British Columbia V6B 5C6 Canada ("Galaxy") and Galaxy Telnet SRL, having a principal office at Aleea Malinului, Nr. 11, Bl. D, Scara C, Apt. 43, Constanta, Judetul Constanta, Romania ("Telnet") as of the 1s t day of June, 2004. WHEREAS: Galaxy is a wholesale provider of Voice over Internet Protocol ("VoIP") telephony and related services and products. Telnet is a provider of VoIP telephony and related services and products to Subscribers, as defined hereafter. Galaxy wishes to provide to Telnet and Telnet wishes to acquire from Galaxy VOIP related services and products from time to time for the purpose of providing them to Telnet\'s existing and future clients. NOW THEREFORE, for good and valuable consideration, the receipt and sufficiency of which is hereby acknowledged by the Parties, the Parties c

In [27]:
def get_date(content):
    doc = nlp(content)
    dates = [X.text for X in doc.ents if X.label_ == 'DATE']
    return '; '.join(set(dates))

In [28]:
get_date(df['text'][0])

'60 days; the month or; additional years; 30 days; the day; seven calendar days; 15 days; year; Monthly; day of June, 2004; 90) days; monthly; 1-8XX; months; Subscriber; 45) days'

> We can see that only using NER's DATE entity does a poor job at extracting dates!

> Another solution would be to use regular expression (regex), but after examining a few of the documents, I found that the documents do not have a cosistent format, and as a result, regex by itself is not a proper tool either.

> A more suitable tool will be the `spaCy’s rule-based matcher engine`, where we can define our custom rules, and access and analyze the surrounding tokens. Another advantage of spaCy’s rule-based matching is that it can be combined with classic regex, thus enhancing the information extraction process.

> We will create a custom fucntion captures the textual patterns around the date in our documents. I found that most dates are given in one the these formats

            March 21, 1981
            as of (on) March 21, 1981
            03/21/1981 or 3/21/81
            as of the (this) 21st day of March, 1981
            on the (this) 21st day of March, 1981

> SpaCy Matcher patterns used to capture the above

            {} ---> An empty dictionary, {}, as a wildcard representing any token
            {'POS': 'PROPN'} ---> to capture proper nouns including month name: "March"
            'OP': '*' ---> Allow the pattern to match zero or more times
            "OP": "?" ---> Make the pattern optional, by allowing it to match 0 or 1 times      

> Patterns used for regex

            'REGEX': '(\w+\s\d{1,2},\s\d{2,4})' ---> to find March 21, 1981
            'REGEX': '\d{1,2}\/\d{1,2}\/\d{2,4}' ---> to find 03/21/1981 or 3/21/81

In [29]:
def get_date(content):
    import spacy
    from spacy.matcher import Matcher

    nlp = spacy.load("en_core_web_sm")
    # create Matcher object
    matcher = Matcher(nlp.vocab)

    patterns = [
    
        # to capture ---> ... March 21, 1981
        [
            {'TEXT': {'REGEX': '(\w+\s\d{1,2},\s\d{2,4})'}}  
        ],
        
        # to capture ---> as of (on) March 21, 1981
        [
            {'POS': 'ADP', 'OP': '*'},    # to get ---> as of (on) or nothing
            {'POS': 'PROPN'},
            {'LIKE_NUM': True},
            {'IS_PUNCT': True},
            {'LIKE_NUM': True}
        ],
        
        # to capture ---> 03/21/1981 or 3/21/81
        [
            {'TEXT': {'REGEX': '\d{1,2}\/\d{1,2}\/\d{2,4}'}}  
        ],
        
        # to capture ---> as of the (this) 21st day of March, 1981
        [
            {'LOWER': 'as'},
            {'LOWER': 'of'},
            {'LOWER': {'IN': ['the', 'this']}},
            {'OP': '*'},
            {'TEXT': 'day'},
            {'TEXT': 'of'},
            {'POS': 'PROPN'},
            {'IS_PUNCT': True},
            {'LIKE_NUM': True}
        ],
        
        # to capture ---> on the (this) 21st day of March, 1981
        [
            {'LOWER': 'on'},
            {'LOWER': {'IN': ['the', 'this']}},
            {'OP': '*'},
            {'LOWER': 'day'},
            {'LOWER': 'of'},
            {'POS': 'PROPN'},
            {'IS_PUNCT': True},
            {'LIKE_NUM': True}
        ]
    ]

    matcher.add("effdate", patterns)

    doc = nlp(content)
    matches = matcher(doc)
    
    previous_temp_date = ''
    if matches != []:
        for match_id, start, end in matches:
            span = doc[start:end]                      # The matched span
            if previous_temp_date != span.text:        # this "if" will only return different span.texts (to ignore duplicates) 
                return span.text
            previous_temp_date = span.text
    else:
        return 'no date found'

In [32]:
df['effective_date'] = df['text'].map(get_date)

In [53]:
df['effective_date'].value_counts()

no date found                30
8/14/2015                     4
of September 30, 1998         2
of September 30, 2019         2
4/30/97                       2
                             ..
of August 9, 1999             1
of May 1, 2019                1
of October 30, 2019           1
of June 11, 2018              1
into on February 26, 1999     1
Name: effective_date, Length: 466, dtype: int64

> The dates for 30 contracts were not identified by the algorithm; either becauase the dates are not specified, or becauase they are specified in a different textual structure from what we included in our spaCy’s rule-based matcheing.

# Contract Dollar Value

In [35]:
def get_dollar(content):
    doc = nlp(content)
    amounts = [X.text for X in doc.ents if X.label_ == 'MONEY']
    return '; '.join(set(amounts))

In [36]:
get_dollar(df['text'][15])

'250,000; Termination; 4,500,000; Fifty Thousand Dollars'

> Again, NER MONEY algorithm extracts information that in some cases is not 100% money related (Example above). So, we will use `spaCy’s rule-based matcher engine`, and limit our search to amounts that are preceded by $ in the contract.

In [37]:
def get_dollar(content):
    import spacy
    from spacy.matcher import Matcher

    nlp = spacy.load("en_core_web_sm")
    # create Matcher object
    matcher = Matcher(nlp.vocab)

    patterns = [[{'ORTH': '$'}, {'LIKE_NUM': True}]]

    matcher.add("effdate", patterns)

    doc = nlp(content)
    matches = matcher(doc)

    previous_temp_date = ''
    if matches != []:
        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            if previous_temp_date != span.text:  # this "if" will only return different span.texts (to ignore duplicates)
                return span.text
            previous_temp_date = span.text
    else:
        return 'no dollar amount found'

In [38]:
df['contract_dollar_value'] = df['text'].map(get_dollar)

In [40]:
df['contract_dollar_value'].value_counts()

no dollar amount found    248
$1,000,000                 17
$500,000                    9
$5,000,000                  9
$0.01                       8
                         ... 
$18,000                     1
$2,000,000.00               1
$50                         1
$9,000,000                  1
$480,000.00                 1
Name: contract_dollar_value, Length: 156, dtype: int64

In [43]:
df

text  \
0    Exhibit 10.5 Reseller Agreement This Agreement...   
1    AMENDMENT NO. 3 Dated as of February 19, 2007 ...   
2    Exhibit 10.26   EXECUTION VERSION   CONFIDENTI...   
3    Exhibit 99.4 JOINT FILING AGREEMENT In accorda...   
4    Exhibit 10.6 ATTACHMENT A ERCHONIA CORPORATION...   
..                                                 ...   
505  1                                             ...   
506  Exhibit 10.16   ENDORSEMENT AGREEMENT     THIS...   
507  Exhibit 10.1 CERTAIN CONFIDENTIAL PORTIONS OF ...   
508  Strategic Alliance Agreement   This agreement ...   
509  1                                             ...   

                   contract_countries  \
0                     Romania, Canada   
1                                       
2    China, Cayman Islands, Hong Kong   
3                                       
4                       United States   
..                                ...   
505                                     
506                     United States   
507                           Bermuda   
508                                     
509                                     

                                       contract_cities  \
0                                    Vancouver, Canada   
1                                                        
2    Beijing, Law, Republic, Hong Kong, Agency, Chi...   
3                                             Delaware   
4       Melbourne, California, Roseville, Florida, Usa   
..                                                 ...   
505                                            Florida   
506                                            America   
507       Watford, Fleet, Purchase, Canon, England, Eu   
508                   Rancho Cordova, Nevada, Colorado   
509                Irvine, California, San Luis Obispo   

                              contract_cities_clean  \
0                                         Vancouver   
1                                                     
2        Beijing, Law, Republic, Agency, May, Macau   
3                                          Delaware   
4    Melbourne, California, Roseville, Florida, Usa   
..                                              ...   
505                                         Florida   
506                                         America   
507    Watford, Fleet, Purchase, Canon, England, Eu   
508                Rancho Cordova, Nevada, Colorado   
509             Irvine, California, San Luis Obispo   

                            contract_cities_final  \
0                                       Vancouver   
1                                                   
2      Beijing, Law, Republic, Agency, May, Macau   
3                                        Delaware   
4       Melbourne, California, Roseville, Florida   
..                                            ...   
505                                       Florida   
506                                                 
507  Watford, Fleet, Purchase, Canon, England, Eu   
508              Rancho Cordova, Nevada, Colorado   
509           Irvine, California, San Luis Obispo   

                                               parties  \
0                        Galaxy Telnet SRL - B.C. Ltd.   
1    Glu Mobile Inc. - Administrator for Twentieth ...   
2    Beijing SINA Internet Information Service Co.,...   
3                                                   NA   
4    Second Floor - Erchonia Corporation - InnerSco...   
..                                                 ...   
505  Sykes HealthPlan Services, Inc. - HealthPlan S...   
506                     GENO AURIEMMA - BERKSHIRE BANK   
507  Bermuda) Ltd. - Canon's Court - Fleet Laborato...   
508              Fazync LLC - Bravatek Solutions, Inc.   
509  TrueLink, Inc. - Two Venture Plaza - Mortgage ...   

                       effective_date   contract_dollar_value  
0    as of the 1s t day of June, 2004                   $9.95  
1                of February 19, 2007  n